In [2]:
import numpy as np
import os
import sys
import tensorflow as tf

sys.path.append("..")
from utils import label_map_util
# from utils import visualization_utils as vis_util

########## For TF2.1 ############
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#################################

import cv2
from time import time

In [3]:
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(r"D:\Courses\pyimagesearch\real-time object detection\real-time-object-detection\external data\aerialfootage.mp4")

Choose a Model

In [4]:
# MODEL_NAME = 'ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03'
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
# MODEL_NAME = 'ssd_mobilenet_v2_oid_v4_2018_12_12'
# MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09'
MODEL_NAME = 'trt_ssdlite'
# MODEL_NAME = 'trt_trial'

In [5]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [6]:
# ## Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, 
# we know that this corresponds to `airplane`.  Here we use internal utility functions, 
# but anything that returns a dictionary mapping integers to appropriate string labels would be fine
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [8]:
np.random.seed(2)
COLORS = np.random.randint(0, 255, size=(90, 3), dtype="uint8")

def draw_outputs(img, outputs, class_names):
    boxes, objectness, classes, nums = outputs
    h, w = img.shape[0:2]
    for i in range(nums):
        if objectness[i] > 0.3:
            top, left, bottom, right = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]

            x1y1 = (int(w*left), int(h*top))
            x2y2 = (int(w*right), int(h*bottom))
            color = [int(c) for c in COLORS[classes[i]]]
            img = cv2.rectangle(img, x1y1, x2y2, color, 4)
            img = cv2.putText(img, '{} {:.4f}'.format(
                class_names[classes[i]]['name'], objectness[i]),
                x1y1, cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (255, 255, 255), 1)
    return img

In [9]:
times = []

viz_times = []
afps_times = []

# Detection
with tf.device('/gpu:0'):
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            while True:
                t1_afps = time()
                # Read frame from camera
                ret, image_np = cap.read()
                
                if not ret:
                    print('cap.read() did not return image\n')
                    break
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                # Extract image tensor
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Extract detection boxes
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Extract detection scores
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                # Extract detection classes
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                # Extract number of detectionsd
                num_detections = detection_graph.get_tensor_by_name(
                  'num_detections:0')
                
                # Actual detection.
                t1 = time()
                
                (boxes, scores, classes, num_detections) = sess.run(
                  [boxes, scores, classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
                
                times.append(time() - t1)
                
                t1_viz = time()
                # Visualization of the results of a detection.
                
#                 vis_util.visualize_boxes_and_labels_on_image_array(
#                   image_np,
#                   np.squeeze(boxes),
#                   np.squeeze(classes).astype(np.int32),
#                   np.squeeze(scores),
#                   category_index,
#                   use_normalized_coordinates=True,
#                   line_thickness=4)
                
                boxes_squeezed = np.squeeze(boxes)
                nums = len(boxes_squeezed[boxes_squeezed != 0])    # Number of detected objects
                img = draw_outputs(image_np, (boxes_squeezed,
                                              np.squeeze(scores), 
                                              np.squeeze(classes).astype(np.int32), 
                                              nums), category_index)
                
                viz_times.append(time() - t1_viz)
                
                # Display output
                image_np = cv2.putText(image_np, "Time: {:,.2f}ms | FPS: {:.2f}fps".format(times[-1]*1000, 1/times[-1]),
                                       (0, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
                
#                 cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))
                cv2.imshow('object detection', image_np)
                afps_times.append(time() - t1_afps)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                  break


# cap.release()
cv2.destroyAllWindows()

if len(times) > 0:
    print('Average FPS: {:,.2f}fps'.format(1/(sum(times)/len(times))))
    print('Average AFPS: {:,.2f}fps'.format(1/(sum(afps_times)/len(afps_times))))

Average FPS: 63.14fps
Average AFPS: 29.20fps


In [10]:
print('average time to vizualize: {:,.2f}ms'.format(sum(viz_times)*1000/len(viz_times)))

average time to vizualize: 0.18ms
